## Imports

In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from ipaddress import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout, Flatten, SpatialDropout1D, LSTM,Input, concatenate, Reshape, InputLayer, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.regularizers import l1, l2, l1_l2
import re
import tldextract
import warnings
warnings.filterwarnings("ignore")
sns.set_style(style='white') 
from sklearn.svm import LinearSVC

## Data Transforming

In [24]:

data=pd.read_csv('cleaned_data.csv')
df=data[['url','status']]
data=data.drop('url',axis=1)

In [25]:
data

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,num_subdomains,subdomain_digits,is_HTTPS,depth,top_domain_encoded,domain_encoded,subdomain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,7,0,1,225,111707,17
1,0,36,0,6,2,0,0,0,12,0,1,6,0,1,225,111707,18
2,0,12,0,5,1,0,0,0,5,5,0,0,0,1,878,10,0
3,0,35,0,4,2,0,0,0,12,0,1,4,0,1,225,111707,25
4,0,20,0,5,2,0,0,0,8,0,1,5,0,1,225,291146,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798458,0,9,0,0,1,0,0,0,5,0,0,0,0,1,225,308751,0
798459,0,6,0,0,1,0,0,0,3,0,0,0,0,1,690,308750,0
798460,0,9,0,0,2,0,0,0,3,0,0,0,0,1,215,308771,0
798461,0,16,0,0,2,0,0,0,10,0,0,0,0,1,219,308776,0


In [26]:
a=data.top_domain_encoded.nunique()
data.top_domain_encoded.nunique()

1232

In [27]:
b=data.domain_encoded.nunique()
data.domain_encoded.nunique()

308783

In [28]:
c=data.subdomain_encoded.nunique()
data.subdomain_encoded.nunique()

111203

In [29]:
d=a+b+c
d

421218

In [30]:
data.head()

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,num_subdomains,subdomain_digits,is_HTTPS,depth,top_domain_encoded,domain_encoded,subdomain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,7,0,1,225,111707,17
1,0,36,0,6,2,0,0,0,12,0,1,6,0,1,225,111707,18
2,0,12,0,5,1,0,0,0,5,5,0,0,0,1,878,10,0
3,0,35,0,4,2,0,0,0,12,0,1,4,0,1,225,111707,25
4,0,20,0,5,2,0,0,0,8,0,1,5,0,1,225,291146,29


In [31]:

y=data['status']
X=data.drop('status', axis=1)
# col=X.columns
# X=StandardScaler().fit_transform(X)
# X=pd.DataFrame(X,columns=col)

In [32]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=54)

In [33]:
X_train.shape

(598847, 16)

In [34]:
X_test.shape

(199616, 16)

In [35]:
X_train

,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,num_subdomains,subdomain_digits,is_HTTPS,depth,top_domain_encoded,domain_encoded,subdomain_encoded
248801,46,6,0,2,2,0,0,11,0,0,0,0,2,225,237481,0
213106,128,1,7,16,2,0,0,16,0,13,7,0,3,225,49622,66139
556820,74,7,0,1,2,0,0,14,0,0,0,0,3,804,173171,0
662529,33,0,1,1,3,0,0,14,0,0,0,0,3,69,21404,0
427785,103,1,3,2,2,0,0,7,0,1,0,0,3,225,22491,103930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224993,23,0,0,2,2,0,0,8,0,0,0,0,2,225,87508,0
267287,37,0,0,3,2,0,0,8,0,1,0,0,2,138,170447,46165
323343,41,0,0,2,2,0,0,8,0,1,0,0,3,225,158881,19396
367941,66,1,2,2,2,0,0,5,0,1,0,0,3,225,294355,46689


## Traditional Models

In [36]:
# rf = RandomForestClassifier()
# rf.fit(X_train, y_train)
# y_pred = rf.predict(X_test)
# print(classification_report(y_test, y_pred))
# trains in around 2 minutes and has 0.95 across the board

In [37]:
# vectorizer = TfidfVectorizer(analyzer='char')
# X=vectorizer.fit_transform(df.url)
# tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
# y1=df['status']
# X1=tfidf_df
# X_train1, X_test1, y_train1,y_test1 = train_test_split(X,y,test_size=0.2,random_state=504)

# rf = RandomForestClassifier()
# rf.fit(X_train1, y_train1)
# y_pred1 = rf.predict(X_test1)
# print(classification_report(y_test1, y_pred1))


In [38]:
# clf = LinearSVC() 

# clf.fit(X_train, y_train)

# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))
# ran for 5 minutes and did not finish training

In [39]:

# clf = SGDClassifier() 

# clf.fit(X_train, y_train)

# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))
# took  4 minuted but terrible results

# Neural Networks

In [40]:
tld_num=data['top_domain_encoded'].nunique()
ld_num=data['domain_encoded'].nunique()
sub_num=data['subdomain_encoded'].nunique()



## LSTM

In [41]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)


In [42]:
Lstm  = Sequential()
Lstm.add(Embedding(d,8))
Lstm.add(Dropout(0.2))
Lstm.add(LSTM(128,return_state=False))
Lstm.add(Dense(24,activation='relu'))
# Lstm.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01)))
Lstm.add(Dense(1,activation ='sigmoid'))

In [43]:
Lstm.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [44]:
num_epochs =30
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyL=Lstm.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)
# around8 minutes an epoch when split 

Epoch 1/30
18714/18714 - 566s - 30ms/step - accuracy: 0.9285 - loss: 0.1681 - precision: 0.9253 - recall: 0.9423 - val_accuracy: 0.9576 - val_loss: 0.1035 - val_precision: 0.9597 - val_recall: 0.9610
Epoch 2/30
18714/18714 - 576s - 31ms/step - accuracy: 0.9820 - loss: 0.0512 - precision: 0.9832 - recall: 0.9832 - val_accuracy: 0.9497 - val_loss: 0.1339 - val_precision: 0.9594 - val_recall: 0.9460
Epoch 3/30
18714/18714 - 573s - 31ms/step - accuracy: 0.9904 - loss: 0.0294 - precision: 0.9907 - recall: 0.9913 - val_accuracy: 0.9517 - val_loss: 0.1441 - val_precision: 0.9547 - val_recall: 0.9549
Epoch 4/30
18714/18714 - 569s - 30ms/step - accuracy: 0.9931 - loss: 0.0213 - precision: 0.9930 - recall: 0.9941 - val_accuracy: 0.9534 - val_loss: 0.1515 - val_precision: 0.9553 - val_recall: 0.9576
Epoch 4: early stopping
Restoring model weights from the end of the best epoch: 1.


## CNN

In [45]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)

In [46]:
cnn  = Sequential()
cnn.add(Embedding(d,8))
cnn.add(Dropout(0.2))
cnn.add(Conv1D(128, 5, activation='relu'))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(24,activation='relu'))
# cnn.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01)))
cnn.add(Dense(1,activation ='sigmoid'))

In [47]:
cnn.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [48]:
num_epochs =30
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyC=cnn.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)
# around 6 minutes an epoch when split 

Epoch 1/30


18714/18714 - 255s - 14ms/step - accuracy: 0.9402 - loss: 0.1437 - precision: 0.9362 - recall: 0.9530 - val_accuracy: 0.9615 - val_loss: 0.0936 - val_precision: 0.9614 - val_recall: 0.9668
Epoch 2/30
18714/18714 - 277s - 15ms/step - accuracy: 0.9883 - loss: 0.0344 - precision: 0.9899 - recall: 0.9883 - val_accuracy: 0.9488 - val_loss: 0.1243 - val_precision: 0.9585 - val_recall: 0.9452
Epoch 3/30
18714/18714 - 336s - 18ms/step - accuracy: 0.9952 - loss: 0.0156 - precision: 0.9958 - recall: 0.9951 - val_accuracy: 0.9516 - val_loss: 0.1360 - val_precision: 0.9420 - val_recall: 0.9692
Epoch 4/30
18714/18714 - 311s - 17ms/step - accuracy: 0.9966 - loss: 0.0113 - precision: 0.9969 - recall: 0.9967 - val_accuracy: 0.9507 - val_loss: 0.1320 - val_precision: 0.9415 - val_recall: 0.9678
Epoch 4: early stopping
Restoring model weights from the end of the best epoch: 1.
